


# 基于 Bert 做中文文本分类




## 连接 Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
cd /content/drive/MyDrive/NLP_study/transformers_wp/bert_classifier/

In [ ]:
%tensorflow_version 1.x

In [ ]:
!pip install urllib3==1.25.4

In [ ]:
!pip install pytorch_pretrained_bert

## 加载库

In [ ]:
import os
import sys
import pickle
from tqdm import tqdm
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
import pandas as pd
import numpy as np
from concurrent.futures import ThreadPoolExecutor
import torch
from sklearn.preprocessing import LabelEncoder
from torch.optim import optimizer
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
from torch.nn import CrossEntropyLoss,BCEWithLogitsLoss
from tqdm import tqdm_notebook, trange
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM, BertForSequenceClassification
from pytorch_pretrained_bert.optimization import BertAdam, WarmupLinearSchedule
from sklearn.metrics import precision_recall_curve,classification_report
import matplotlib.pyplot as plt
import torch.nn as nn
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
%matplotlib inline

## 工具类

In [ ]:
# 功能：字典数据 存储 为 plk
def save_plk_dict(dic,save_path,fila_name):
    '''
        功能：字典数据 存储 为 plk
        input:
        dic          Dict     存储字典    
        save_path     String    存储目录 
        fila_name     String    存储文件 
    return:
        
    '''
    with open(save_path+ fila_name + '.pkl', 'wb') as f:
        pickle.dump(dic, f, pickle.HIGHEST_PROTOCOL) 

def load_plk_dict(save_path,fila_name):
    '''
        功能：加载 plk 中 字典数据
        input: 
        save_path     String    存储目录 
        fila_name     String    存储文件 
    return:
        dic        Dict     字典数据 
        
    '''
    with open(save_path+ fila_name + '.pkl', 'rb') as f:
        return pickle.load(f)      

## 参数初始化

In [ ]:
class Config():
  def __init__(self):
    self.split_ratio = 0.9   #训练和验证集的比例
    #MAX_SEQ_LEN = 50
    self.batch_size = 128
    self.seed = 0
    self.epochs = 2
    self.num_labels = 10
    self.is_colab = True
    if self.is_colab:
      self.data_path = "data/"
      self.bert_model_path = ".././../../bert_series/bert/bert-chinese-wwm_torch/"
    else:
      self.data_path = "F:/document/datasets/nlpData/text_classifier_data/THUCNews_ch/"
      self.bert_model_path = "F:/document/datasets/nlpData/pretrain/bert/chinese_wwm_ext_pytorch/"

    self.data_file_list = [
        "cnews.train","cnews.val","cnews.test"
    ]

    self.data_index = 2
    self.patience = 20
    self.do_lower_case = False
    self.device = torch.device('cpu' if not torch.cuda.is_available() else "cuda")
    self.is_demo = True

config = Config()

## 早停法

In [ ]:
#早停法
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        # torch.save(model.state_dict(), 'checkpoint_loss.pt')
        torch.save(model, open("checkpoint_loss.bin", "wb"))
        self.val_loss_min = val_loss

## Label Smoothing（标签平滑法）

In [ ]:
#标签平滑
class LabelSmoothing(nn.Module):
    def __init__(self, size, smoothing=0.0):
        super(LabelSmoothing, self).__init__()
        self.criterion = nn.KLDivLoss(size_average=False)
        #self.padding_idx = padding_idx
        self.confidence = 1.0 - smoothing#if i=y的公式
        self.smoothing = smoothing
        self.size = size
        self.true_dist = None
    
    def forward(self, x, target):
        """
        x表示输入 (N，M)N个样本，M表示总类数，每一个类的概率log P
        target表示label（M，）
        """
        assert x.size(1) == self.size
        true_dist = x.data.clone()#先深复制过来
        #print true_dist
        true_dist.fill_(self.smoothing / (self.size - 1))#otherwise的公式
        #print true_dist
        #变成one-hot编码，1表示按列填充，
        #target.data.unsqueeze(1)表示索引,confidence表示填充的数字
        true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        self.true_dist = true_dist

        return self.criterion(x, Variable(true_dist, requires_grad=False))

## 数据预处理

### 数据处理类

In [ ]:
# 数据处理类
class DataPrecessForSingleSentence(object):
    """
    对文本进行处理
    """

    def __init__(self, bert_tokenizer, max_workers=10):
        """
        bert_tokenizer :分词器
        dataset        :包含列名为'text'与'label'的pandas dataframe
        """
        self.bert_tokenizer = bert_tokenizer
        # 创建多线程池
        self.pool = ThreadPoolExecutor(max_workers=max_workers)
        # 获取文本与标签

    def get_input(self, dataset, max_seq_len=50):
        """
            通过多线程（因为notebook中多进程使用存在一些问题）的方式对输入文本进行分词、ID化、截断、填充等流程得到最终的可用于模型输入的序列。

            入参:
                dataset     : pandas的dataframe格式，包含两列，第一列为文本，第二列为标签。标签取值为{0,1}，其中0表示负样本，1代表正样本。
                max_seq_len : 目标序列长度，该值需要预先对文本长度进行分别得到，可以设置为小于等于512（BERT的最长文本序列长度为512）的整数。

            出参:
                seq         : 在入参seq的头尾分别拼接了'CLS'与'SEP'符号，如果长度仍小于max_seq_len，则使用0在尾部进行了填充。
                seq_mask    : 只包含0、1且长度等于seq的序列，用于表征seq中的符号是否是有意义的，如果seq序列对应位上为填充符号，
                              那么取值为1，否则为0。
                seq_segment : shape等于seq，因为是单句，所以取值都为0。
                labels      : 标签取值为{0,1}，其中0表示负样本，1代表正样本。   
        """
        sentences = dataset.iloc[:, 0].tolist()
        labels = dataset.iloc[:, 1].tolist()
        # 切词
        tokens_seq = list(
            self.pool.map(self.bert_tokenizer.tokenize, sentences))
        # 获取定长序列及其mask
        result = list(
            self.pool.map(self.trunate_and_pad, tokens_seq,
                          [max_seq_len] * len(tokens_seq)))
        seqs = [i[0] for i in result]
        seq_masks = [i[1] for i in result]
        seq_segments = [i[2] for i in result]
        return seqs, seq_masks, seq_segments, labels

    def trunate_and_pad(self, seq, max_seq_len):
        """
            1. 因为本类处理的是单句序列，按照BERT中的序列处理方式，需要在输入序列头尾分别拼接特殊字符'CLS'与'SEP'，
               因此不包含两个特殊字符的序列长度应该小于等于max_seq_len-2，如果序列长度大于该值需要那么进行截断。
            2. 对输入的序列 最终形成['CLS',seq,'SEP']的序列，该序列的长度如果小于max_seq_len，那么使用0进行填充。

            入参: 
                seq         : 输入序列，在本处其为单个句子。
                max_seq_len : 拼接'CLS'与'SEP'这两个特殊字符后的序列长度

            出参:
                seq         : 在入参seq的头尾分别拼接了'CLS'与'SEP'符号，如果长度仍小于max_seq_len，则使用0在尾部进行了填充。
                seq_mask    : 只包含0、1且长度等于seq的序列，用于表征seq中的符号是否是有意义的，如果seq序列对应位上为填充符号，
                              那么取值为1，否则为0。
                seq_segment : shape等于seq，因为是单句，所以取值都为0。
           
        """
        # 对超长序列进行截断
        if len(seq) > (max_seq_len - 2):
            seq = seq[0:(max_seq_len - 2)]
        # 分别在首尾拼接特殊符号
        seq = ['[CLS]'] + seq + ['[SEP]']
        # ID化
        seq = self.bert_tokenizer.convert_tokens_to_ids(seq)
        # 根据max_seq_len与seq的长度产生填充序列
        padding = [0] * (max_seq_len - len(seq))
        # 创建seq_mask
        seq_mask = [1] * len(seq) + padding
        # 创建seq_segment
        seq_segment = [0] * len(seq) + padding
        # 对seq拼接填充序列
        seq += padding
        assert len(seq) == max_seq_len
        assert len(seq_mask) == max_seq_len
        assert len(seq_segment) == max_seq_len
        return seq, seq_mask, seq_segment

### 数据处理函数定义

In [ ]:
# 功能：数据加载函数
def load_data(data_path,data_file):
  '''
    功能：数据加载函数
    input:
      data_path     String      数据目录   
      data_file     String      数据文件名称
    return:
      data        List       数据
      num_labels     int       标签类别数量
  '''
  # 数据加载
  data = pd.read_table(f'{data_path}{data_file}.txt', encoding='utf-8', names=['label', 'text'])
  data = data[['text', 'label']]
  # 标签编码
  le = LabelEncoder()
  le.fit(data.label.tolist())
  data['label_id'] = le.transform(data.label.tolist())
  return data[['text', 'label', 'label_id']]

In [ ]:
train_data = load_data(config.data_path,config.data_file_list[config.data_index])

### 标签映射表构建

In [ ]:
# 功能：标签映射表构建
def build_label2id(data,data_path):
  '''
    功能：标签映射表构建
    input:
      data     DataFrame     训练数据
    return:
      label2id_dic Dict        label 到 id 的映射
      id2label_dic Dict        id 到 label 的映射
  '''
  labeldata = data.groupby(['label', 'label_id']).count().reset_index()
  label2id_dic = {}
  id2label_dic = {}
  for index,row in labeldata.T.iteritems():
    label2id_dic[row['label']] = row['label_id']
    id2label_dic[row['label_id']] = row['label']

  label2id = {
      "label2id":label2id_dic,
      "id2label":id2label_dic
  }

  save_plk_dict(label2id,data_path,"label2id")
  return label2id_dic,id2label_dic
  
label2id_dic,id2label_dic = build_label2id(train_data,config.data_path)
train_data = train_data[['text','label_id']]

### 训练数据集拆分为训练集和验证集

In [ ]:
train, valid = train_test_split(train_data, train_size=config.split_ratio, random_state=config.seed)
if config.is_demo:
  train = train[0:int(0.01*len(train))]
  valid = train[0:int(0.01*len(valid))]
train_labels = train.groupby(['label_id']).count().reset_index()
valid_labels = valid.groupby(['label_id']).count().reset_index()

## bert 模型加载

In [ ]:
# 分词工具
bert_tokenizer = BertTokenizer.from_pretrained(config.bert_model_path, do_lower_case=config.do_lower_case)
# 类初始化
processor = DataPrecessForSingleSentence(bert_tokenizer= bert_tokenizer)
# 加载预训练的bert模型
model = BertForSequenceClassification.from_pretrained(config.bert_model_path, num_labels=config.num_labels)

## 数据编码

In [ ]:
def build_data(processor,data,batch_size):
  # 产生训练集输入数据
  seqs, seq_masks, seq_segments, labels = processor.get_input(dataset=data)
  # 转换为torch tensor
  t_seqs = torch.tensor(seqs, dtype=torch.long)
  t_seq_masks = torch.tensor(seq_masks, dtype = torch.long)
  t_seq_segments = torch.tensor(seq_segments, dtype = torch.long)
  t_labels = torch.tensor(labels, dtype = torch.long)

  data = TensorDataset(t_seqs, t_seq_masks, t_seq_segments, t_labels)
  sampler = RandomSampler(data)
  dataloder = DataLoader(dataset= data, sampler=sampler, batch_size = batch_size)
  return dataloder

In [ ]:
train_dataloder = build_data(processor,train,config.batch_size)
valid_dataloder = build_data(processor,valid,config.batch_size)

## 模型参数设置

In [ ]:
# 待优化的参数
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params':[p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay':0.01},
    {'params':[p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay':0.0}
]

steps = len(train_dataloder) * config.epochs
optimizer = BertAdam(optimizer_grouped_parameters, lr=2e-05, warmup= 0.1 , t_total= steps)
loss_function = LabelSmoothing(config.num_labels, 0.1)

## 模型训练

In [ ]:
model = model.to(config.device)
#存储loss
train_losses = []
valid_losses = []
avg_train_losses = []
avg_valid_losses = []
early_stopping = EarlyStopping(patience=config.patience, verbose=True)
best_f1 = 0
for i in trange(config.epochs, desc='Epoch'): 
    model.train() #训练
    for step, batch_data in enumerate(tqdm(train_dataloder, desc='Train Iteration')):
        batch_data = tuple(t.to(config.device) for t in batch_data)
        batch_seqs, batch_seq_masks, batch_seq_segments, batch_labels = batch_data
        # 对标签进行onehot编码
        if torch.cuda.is_available():
            one_hot = torch.zeros(batch_labels.size(0), config.num_labels).long().cuda()  #gpu版本
            one_hot_batch_labels = one_hot.scatter_(
                dim=1,
                index=torch.unsqueeze(batch_labels, dim=1),
                src=torch.ones(batch_labels.size(0), config.num_labels
            ).long().cuda())
        else:
            # 以下注释为cpu版本
            one_hot = torch.zeros(batch_labels.size(0), config.num_labels).long()   #cpu版本
            one_hot_batch_labels = one_hot.scatter_(
              dim=1,
              index=torch.unsqueeze(batch_labels, dim=1),
              src=torch.ones(batch_labels.size(0), config.num_labels
            ).long())

        logits = model(
            batch_seqs, batch_seq_masks, batch_seq_segments, labels=None)
        logits = torch.nn.functional.log_softmax(logits, dim=1)
        #loss_function = CrossEntropyLoss()
        loss = loss_function(logits, batch_labels)
        loss.backward()
        train_losses.append(loss.item())
        print("\r%f" % loss, end='')
        optimizer.step()
        optimizer.zero_grad()
        
    model.eval() #验证
    true_labels = []
    pred_labels = []
    for step, batch_data in enumerate(
            tqdm(valid_dataloder, desc='Dev Iteration')):
        with torch.no_grad():
            batch_data = tuple(t.to(config.device) for t in batch_data)
            batch_seqs, batch_seq_masks, batch_seq_segments, batch_labels = batch_data
            if torch.cuda.is_available():
                # 对标签进行onehot编码，以下注释为gpu版本
                one_hot = torch.zeros(batch_labels.size(0), config.num_labels).long().cuda()
                one_hot_batch_labels = one_hot.scatter_(
                    dim=1,
                    index=torch.unsqueeze(batch_labels, dim=1),
                    src=torch.ones(batch_labels.size(0), config.num_labels
                ).long().cuda())
            else:
                # cpu
                one_hot = torch.zeros(batch_labels.size(0), config.num_labels).long()
                one_hot_batch_labels = one_hot.scatter_(
                  dim=1,
                  index=torch.unsqueeze(batch_labels, dim=1),
                  src=torch.ones(batch_labels.size(0), config.num_labels
                ).long())

            logits = model(
                batch_seqs, batch_seq_masks, batch_seq_segments, labels=None)
            logits = torch.nn.functional.log_softmax(logits, dim=1)
            loss = loss_function(logits, batch_labels)
            valid_losses.append(loss.item())
            
            logits = logits.softmax(dim=1).argmax(dim = 1)
            pred_labels.append(logits.detach().cpu().numpy())
            true_labels.append(batch_labels.detach().cpu().numpy())
    
    true_labels = np.concatenate(true_labels)
    pred_labels = np.concatenate(pred_labels)
    precision = precision_score(true_labels, pred_labels, average='micro')
    recall = recall_score(true_labels, pred_labels, average='micro')
    f1 = f1_score(true_labels, pred_labels, average='micro')
    
    if best_f1<f1:
      # torch.save(model.state_dict(), 'checkpoint_f1.pt')
      torch.save(model, open("checkpoint_f1.bin", "wb"))

    train_loss = np.average(train_losses)
    valid_loss = np.average(valid_losses)
    avg_train_losses.append(train_loss)
    avg_valid_losses.append(valid_loss)
    print("train_loss:%f, valid_loss:%f, precision:%f, recall:%f, f1:%f " %(train_loss, valid_loss,precision,recall,f1))
    
    #重置训练损失和验证损失
    train_losses = []
    valid_losses = []
    
    early_stopping(valid_loss, model)
    if early_stopping.early_stop:
        print("Early Stopping")
        break

## 绘制loss图

In [ ]:
# 功能：绘制 损失函数 loss 曲线
def draw_loss_pic(avg_train_losses,avg_valid_losses):
  '''
    功能：绘制 损失函数 loss 曲线
    input:
      avg_train_losses
      avg_valid_losses
  '''
  %matplotlib inline
  fig = plt.figure(figsize=(8,6))
  plt.plot(range(1, len(avg_train_losses)+1), avg_train_losses, label='Training Loss')
  plt.plot(range(1, len(avg_valid_losses)+1), avg_valid_losses, label='Validation Loss')

  #find the position of lowest validation loss
  minposs = avg_valid_losses.index(min(avg_valid_losses))+1
  plt.axvline(minposs, linestyle='--', color = 'r', label='Early Stopping Checkpoint')
  plt.xlabel('epochs')
  plt.ylabel('loss')
  plt.grid(True)
  plt.legend()
  plt.tight_layout()
  plt.show()
  fig.savefig('loss_plot.png', bbox_inches='tight')

draw_loss_pic(avg_train_losses,avg_valid_losses)

## 测试数据

### 加载模型

In [ ]:
model = torch.load("checkpoint_f1.bin")

### 测试集加载

In [ ]:
import random

In [ ]:
test_data = load_data(config.data_path,config.data_file_list[config.data_index])
if config.is_demo:
  test_data = test_data.sample(int(0.01*len(test_data)))

In [ ]:
test_data = test_data[['text','label_id']]
test_labels = test_data.groupby(['label_id']).count().reset_index()

### 测试数据构建

In [ ]:
test_dataloder = build_data(processor,test_data,config.batch_size)

In [ ]:
# 用于存储预测标签与真实标签
true_labels = []
pred_labels = []
model.eval()
# 预测
with torch.no_grad():
    for batch_data in tqdm_notebook(test_dataloder, desc = 'TEST'):
        batch_data = tuple(t.to(config.device) for t in batch_data)
        batch_seqs, batch_seq_masks, batch_seq_segments, batch_labels = batch_data        
        logits = model(
            batch_seqs, batch_seq_masks, batch_seq_segments, labels=None)
        logits = logits.softmax(dim=1).argmax(dim = 1)
        pred_labels.append(logits.detach().cpu().numpy())
        true_labels.append(batch_labels.detach().cpu().numpy())
# 查看各个类别的准确率和召回率
result = classification_report(np.concatenate(true_labels), np.concatenate(pred_labels))
print(result)

## 预测

In [ ]:
query_list = [
     "25日股票基金全线受挫 九成半基金跌逾1%全景网3月26日讯 周三开放式基金净值普降，股票型基金全线受挫，九成半基金跌幅超过1%。上证综指昨日收市跌2.00%。据全景网基金统计数据，3月25日统计的229只股票型基金全线下挫，其中219只跌幅在1%以上，占比95.63%。跌幅排名前五位的基金是友邦红利ETF、富国天鼎、宝盈资源优选、德盛红利、易方达深100ETF，增长率分别为-3.03%、-2.70%、-2.51%、-2.44%、-2.42%。华富策略精选、景顺公司治理、长城双动力、荷银合丰稳定、汇丰晋信2026等跌幅较小，均在1%以内。积极配置型基金亦全盘尽墨，中欧新蓝筹、金鹰优选两只基金跌幅超过2%，天治财富增长、华夏回报跌幅在0.5%以内。保守配置型基金中，申万盛利配置最为抗跌，下挫0.20%，国投瑞银融华垫底，跌0.99%。债市方面，上证国债指数昨日涨0.03%，上证企债指数跌0.10%。普通债券型基金仅4只飘红，国投瑞银债券领跑，涨0.03%，中信稳定双利、易方达稳健收益B、易方达稳健收益A也小幅上扬。嘉实多元收益A、嘉实多元收益B、华富收益增强B、华富收益增强A跌幅都超过0.4%。（全景网/陈丹蓉）",
     "古天乐投诉大S不给机会 希望下次再合作(组图)新浪娱乐讯 8月26日，电影《保持通话》举行宣传活动，众演员包括古天乐( 听歌 blog)、徐熙媛(大S)、张嘉辉及导演陈木胜等等均有出席。古天乐笑言这次拍摄多是单独演出，并要一直拿着手机演戏，感觉新鲜又甚具压力，他更表示与大S只有一场对手戏，希望下次能够再有机会合作。TUNGSTAR/文并图 "
]
predict_data = pd.DataFrame([{'text':query, 'label_id':0} for query in query_list])
predict_data

In [ ]:
id2label_dic = load_plk_dict(config.data_path,"label2id")['id2label']
id2label_dic

In [ ]:
predict_dataloder = build_data(processor,predict_data,config.batch_size)

In [ ]:
model.eval()
pred_labels = []
# 预测
with torch.no_grad():
    for batch_data in tqdm_notebook(predict_dataloder, desc = 'Predict'):
        batch_data = tuple(t.to(config.device) for t in batch_data)
        batch_seqs, batch_seq_masks, batch_seq_segments, batch_labels = batch_data        
        logits = model(
            batch_seqs, batch_seq_masks, batch_seq_segments, labels=None)
        logits = logits.softmax(dim=1).argmax(dim = 1)
        pred_labels.append(logits.detach().cpu().numpy())
        

In [ ]:
pred_labels